In [ ]:
from cartopy import crs as ccrs
import iris
import iris.plot as iplt
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

from cube_browser import Browser, Pcolormesh

In [ ]:
cube = iris.load_cube(iris.sample_data_path('colpex.pp'), 'air_potential_temperature')

In [ ]:
print cube

In [ ]:
def callback1(axes, plot, slider):
    name, index = slider
    cube = plot.cube
    glon = cube.coord('grid_longitude')
    glat = cube.coord('grid_latitude')
    mln = plot.subcube.coord('model_level_number').points[0]
    xmin, xmax = glon.points[0], glon.points[-1]
    ymin, ymax = glat.points[0], glat.points[-1]
    plt.sca(axes)
    projection = iplt.default_projection(cube)
    title = cube.name().replace('_', ' ').capitalize()
    axes.set_title('{} @ mln {}'.format(title, mln), fontsize=11)
    if name == 'grid_latitude':
        y = glat.points[index]
        plt.plot([xmin, xmax], [y, y], transform=projection, color='black', linewidth=2)
    else:
        x = glon.points[index]
        plt.plot([x, x], [ymin, ymax], transform=projection, color='black', linewidth=2)

In [ ]:
def callback2(axes, plot, slider):
    name, index = slider
    cube = plot.cube
    glon = cube.coord('grid_longitude')
    glat = plot.subcube.coord('grid_latitude').points[0]
    mln = cube.coord('model_level_number')
    xmin, xmax = glon.points[0], glon.points[-1]
    zmin, zmax = mln.points[0] - 0.5, mln.points[-1] + 0.5
    z = mln.points[index]
    plt.sca(axes)
    title = cube.name().replace('_', ' ').capitalize()
    axes.set_title('{} @ glat {:.2f}'.format(title, glat), fontsize=11)
    axes.set_xlabel('Grid Longitude / {}'.format(glon.units))
    axes.set_ylabel('Model Level Number')
    axes.set_xlim([xmin, xmax])
    axes.set_ylim([zmin, zmax])
    axes.set_aspect((xmax - xmin) / (zmax - zmin))
    xaxis = axes.get_xaxis()
    xaxis.set_major_locator(ticker.MaxNLocator(5))
    xaxis.set_major_formatter(ticker.FormatStrFormatter('%0.1f'))
    plt.plot([xmin, xmax], [z, z], color='black', linewidth=2)

In [ ]:
def callback3(axes, plot, slider):
    name, index = slider
    cube = plot.cube
    glon = plot.subcube.coord('grid_longitude').points[0]
    glat = cube.coord('grid_latitude')
    mln = cube.coord('model_level_number')
    ymin, ymax = glat.points[0], glat.points[-1]
    zmin, zmax = mln.points[0] - 0.5, mln.points[-1] + 0.5
    z = mln.points[index]
    plt.sca(axes)
    title = cube.name().replace('_', ' ').capitalize()
    axes.set_title('{} @ glon {:.2f}'.format(title, glon), fontsize=11)
    axes.set_xlabel('Grid Latitude / {}'.format(glat.units))
    axes.set_ylabel('Model Level Number')
    axes.set_xlim([ymin, ymax])
    axes.set_ylim([zmin, zmax])
    axes.set_aspect((ymax - ymin) / (zmax - zmin))
    xaxis = axes.get_xaxis()
    xaxis.set_major_locator(ticker.MaxNLocator(5))
    xaxis.set_major_formatter(ticker.FormatStrFormatter('%0.1f'))
    plt.plot([ymin, ymax], [z, z], color='black', linewidth=2)

In [ ]:
projection = iplt.default_projection(cube)

vmin, vmax = 277.0, 290.0

plt.figure(figsize=(9, 7))
ax1 = plt.subplot(221, projection=projection)
ax2 = plt.subplot(223)
ax3 = plt.subplot(224)

title = '{} / {}'.format(cube.name().replace('_', ' ').capitalize(), cube.units)
sm = plt.cm.ScalarMappable(norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm.set_array([])
plt.subplots_adjust(bottom=0.2, left=0.1, right=0.9)
cax = plt.axes([0.1, 0.08, 0.8, 0.025])
plt.colorbar(sm, cax=cax, label=title, orientation='horizontal')

cf1 = Pcolormesh(cube, ax1, vmin=vmin, vmax=vmax)
coords = ('grid_longitude', 'model_level_number')
cf2 = Pcolormesh(cube, ax2, coords=coords, vmin=vmin, vmax=vmax)
coords = ('grid_latitude', 'model_level_number')
cf3 = Pcolormesh(cube, ax3, coords=coords, vmin=vmin, vmax=vmax)

br = Browser([cf1, cf2, cf3])
br.on_change(ax1, callback1, sliders=['grid_latitude', 'grid_longitude'])
br.on_change(ax2, callback2, sliders='model_level_number')
br.on_change(ax3, callback3, sliders='model_level_number')
br.display()